In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer

In [3]:
import transformers

In [4]:
model_id="./results_mark4/checkpoint-225"

In [5]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer="unsloth/Meta-Llama-3.1-8B-Instruct",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    return_full_text=False
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
q="ถ้ามีคนประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตต้องระวางโทษอย่างไร"
source_information="""- พระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546 มาตรา 132 ผู้ใดประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตหรือไม่ได้จดทะเบียนตามมาตรา 54 ต้องระวางโทษจำคุกไม่เกินสามปี หรือปรับไม่เกินสามแสนบาท หรือทั้งจำทั้งปรับ และปรับอีกไม่เกินวันละหนึ่งหมื่นบาทตลอดเวลาที่ยังฝ่าฝืน"""
content=f"""Query: {q}\nContinue to answer the query by using the Search Results:\n{source_information}"""

In [7]:
messages=[{"role":"user","content":content}]
prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipeline(prompt, max_new_tokens=512, do_sample=False)  # Don't change do_sample=False

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [8]:
# pipeline.tokenizer("แมวกินปลา")

In [9]:
messages

[{'role': 'user',
  'content': 'Query: ถ้ามีคนประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตต้องระวางโทษอย่างไร\nContinue to answer the query by using the Search Results:\n- พระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546 มาตรา 132 ผู้ใดประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตหรือไม่ได้จดทะเบียนตามมาตรา 54 ต้องระวางโทษจำคุกไม่เกินสามปี หรือปรับไม่เกินสามแสนบาท หรือทั้งจำทั้งปรับ และปรับอีกไม่เกินวันละหนึ่งหมื่นบาทตลอดเวลาที่ยังฝ่าฝืน'}]

In [10]:
print(prompt)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Query: ถ้ามีคนประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตต้องระวางโทษอย่างไร
Continue to answer the query by using the Search Results:
- พระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546 มาตรา 132 ผู้ใดประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตหรือไม่ได้จดทะเบียนตามมาตรา 54 ต้องระวางโทษจำคุกไม่เกินสามปี หรือปรับไม่เกินสามแสนบาท หรือทั้งจำทั้งปรับ และปรับอีกไม่เกินวันละหนึ่งหมื่นบาทตลอดเวลาที่ยังฝ่าฝืน<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [11]:
outputs

[{'generated_text': 'ตามพระราชบัญญัติสัญญาซื้อขายล่วงหน้า พ.ศ. 2546 มาตรา 132 ผู้ประกอบกิจการในลักษณะเป็นศูนย์ซื้อขายสัญญาซื้อขายล่วงหน้าโดยไม่ได้รับใบอนุญาตหรือไม่ได้จดทะเบียนตามมาตรา 54 จะต้องระวางโทษจำคุกไม่เกิน 3 ปี หรือปรับไม่เกิน 300,000 บาท หรือทั้งจำทั้งปรับ และปรับอีกไม่เกิน 1,000 บาทต่อวันตลอดเวลาที่ยังฝ่าฝืน'}]